<a href="https://colab.research.google.com/github/Manan1811/FaceNet-Model/blob/main/Copy_of_FaceNet_Model2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [ ]:
import cv2

In [ ]:
!wget https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/wiki_crop.tar

--2020-10-16 18:30:05--  https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/wiki_crop.tar
Resolving data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)... 129.132.52.162
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.162|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 811315200 (774M) [application/x-tar]
Saving to: ‘wiki_crop.tar’

wiki_crop.tar       100%[===================>] 773.73M  10.5MB/s    in 74s     

2020-10-16 18:31:21 (10.4 MB/s) - ‘wiki_crop.tar’ saved [811315200/811315200]



In [ ]:
!wget https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/imdb_crop.tar

--2020-10-16 18:31:21--  https://data.vision.ee.ethz.ch/cvl/rrothe/imdb-wiki/static/imdb_crop.tar
Resolving data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)... 129.132.52.162
Connecting to data.vision.ee.ethz.ch (data.vision.ee.ethz.ch)|129.132.52.162|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7012157440 (6.5G) [application/x-tar]
Saving to: ‘imdb_crop.tar’

imdb_crop.tar       100%[===================>]   6.53G  10.6MB/s    in 11m 1s  

2020-10-16 18:42:24 (10.1 MB/s) - ‘imdb_crop.tar’ saved [7012157440/7012157440]



In [ ]:
!tar -xf wiki_crop.tar
!tar -xf imdb_crop.tar

In [ ]:
import scipy.io
imdbMat = scipy.io.loadmat('imdb_crop/imdb.mat')
imdbPlace = imdbMat['imdb'][0][0]
imdbMat

In [ ]:
import os
place = imdbPlace
where='imdb_crop'
img_loc=[]
corr_ages=[]
total = 0

for i in range(460723):
  #print(place[0][0][i])
  bYear = int(place[0][0][i]/365) #birth year
  #print(bYear)
  taken = place[1][0][i] #photo taken
  #print(taken)
  path = place[2][0][i][0]
  age = taken - bYear
  img_loc.append(os.path.join(where,path))
  corr_ages.append(age)
  """print("AGE", age)
  print('----------------------')
  
  
  faceScore = str(faceScore)
  secFaceScore = str(secFaceScore)
  
  
  
  
  if 'n' not in faceScore: # n as in Inf; if true, implies that there isn't a face in the image
    
    if 'a' in secFaceScore: #a as in NaN; implies that no second face was found
      
      if age >= 0: 
      
        try:
          gender = int(gender)
          total +=1
          
          if i > 1500:
            
            print('----------------------')
            print(i)
            print(bYear)
            print(taken)
            print("AGE", age)
            print("NAME", name)
            print("GENDER", gender)
            print(faceBox)
            print(faceScore)
            print(secFaceScore)
            
            imShow(os.path.join(where,path))
            break
               
        except:
          print('Failed with gender')
          continue"""

#print(total)

In [ ]:
import pandas as pd
df = pd.DataFrame(img_loc,columns=['Image Location'])

In [ ]:
df

In [ ]:
df['Age']=corr_ages

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import cv2
image=cv2.imread(df['Image Location'][10])
new_image=cv2.resize(image,(224,224),interpolation = cv2.INTER_CUBIC)
print(new_image.shape)

(224, 224, 3)


In [ ]:
fig = plt.gcf()
fig.set_size_inches(18, 10)
plt.axis("off")
#plt.rcParams['figure.figsize'] = [10, 5]
plt.imshow(cv2.cvtColor(new_image, cv2.COLOR_BGR2RGB))

In [ ]:
from PIL import Image
import numpy as np
import random
def get_images(batch_size, img_size=(160,160),add=df['Image Location'], age=df['Age']):
  rand=random.sample(range(0, len(add)), batch_size)
  X=[]
  y=[]
  for i in rand:
    image=cv2.imread(add[i])
    curr_img=cv2.resize(image,img_size,interpolation = cv2.INTER_CUBIC)
    curr_img=curr_img.astype('float64')
    curr_img=curr_img/127.5
    curr_img=curr_img-1
    curr_age=age[i]
    X.append(curr_img)
    y.append(curr_age)
  
  return X,y

In [ ]:
! git clone https://github.com/arshagarwal/outlier-experiment.git -b slim_dataset_creation

Cloning into 'outlier-experiment'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 11749 (delta 6), reused 8 (delta 2), pack-reused 11736
Receiving objects: 100% (11749/11749), 147.41 MiB | 17.14 MiB/s, done.
Resolving deltas: 100% (135/135), done.


In [ ]:
cd './outlier-experiment'

/content/outlier-experiment


In [ ]:
! bash import_weights.sh

In [ ]:
from tensorflow.keras.models import load_model
prev_model=load_model('Facenet/facenet_model.h5')
prev_model.load_weights('Facenet/facenet_weights.h5')
prev_model.summary()

In [ ]:
input = np.ones(shape=(1,160,160,3))
prev_model(input).shape

TensorShape([1, 128])

In [ ]:
# Regression Model
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Dropout
model=Sequential()
model.add(Dense(64,activation='relu',input_shape=(128,)))
model.add(BatchNormalization())
model.add(Dense(32,activation='relu'))
model.add(BatchNormalization())
model.add(Dense(16,activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1, activation='linear'))
model.summary()


In [ ]:
model.compile(loss='mean_absolute_error', metrics=['accuracy'], optimizer='adam')

In [ ]:
input = np.ones(shape=(1,128))
print( model(input) )

tf.Tensor([[0.16518433]], shape=(1, 1), dtype=float32)


In [ ]:
len(df)

In [ ]:
cd ..

/content


In [ ]:
length = 460723
batch_size = 50
n_batches = length//batch_size
epochs = 1
iters = epochs * n_batches
for i in range(iters):
  X,Y=get_images(batch_size)
  X=np.array(X)
  Y=np.array(Y)
  X = prev_model(X)
  assert X.shape == (batch_size,128),'expected shape {} O/p shape {}'.format((batch_size, 128), X.shape)
  model.fit(X, Y, batch_size, 1)
  if i%100 == 0:
    print("No of epochs {}".format(i))



In [ ]:
!touch download.sh

'/content'

In [ ]:
from PIL import Image
curr_img = Image.open('/content/data/celeba_hq/train/male/000016.jpg')
curr_img = curr_img.resize((160,160))
curr_img = np.asarray(curr_img)
print(curr_img.shape)

(160, 160, 3)


In [ ]:
import os
from PIL import Image
path = '/content/data/celeba_hq/train/male/'
names = os.listdir(path)
Add = []
Age = []
for i in names:
  curr_img = Image.open(path+i)
  #print(path+i)
  curr_img = curr_img.resize((160,160))
  curr_img = np.asarray(curr_img)
  curr_img = curr_img.astype('float64')
  curr_img /= 127.5
  curr_img = curr_img-1
  X = [curr_img]
  X = np.asarray(X)
  assert X.shape == (1, 160, 160, 3), 'check input image shape'
  X = prev_model(X)
  y = model(X)
  Add.append(i)
  Age.append(y)
  


In [ ]:
for i in range(len(Age)):
   Age[i] = Age[i].numpy().item()

In [ ]:
(int)(Age[0])

26

In [ ]:
!mkdir Face-hq

In [ ]:
! rm -r data

In [ ]:
!bash download.sh celeba-hq-dataset 

Streaming output truncated to the last 5000 lines.
  inflating: ./data/celeba_hq/train/female/174787.jpg  
  inflating: ./data/celeba_hq/train/female/174791.jpg  
  inflating: ./data/celeba_hq/train/female/174799.jpg  
  inflating: ./data/celeba_hq/train/female/174833.jpg  
  inflating: ./data/celeba_hq/train/female/174834.jpg  
  inflating: ./data/celeba_hq/train/female/174839.jpg  
  inflating: ./data/celeba_hq/train/female/174861.jpg  
  inflating: ./data/celeba_hq/train/female/174868.jpg  
  inflating: ./data/celeba_hq/train/female/174874.jpg  
  inflating: ./data/celeba_hq/train/female/174881.jpg  
  inflating: ./data/celeba_hq/train/female/174889.jpg  
  inflating: ./data/celeba_hq/train/female/174890.jpg  
  inflating: ./data/celeba_hq/train/female/174891.jpg  
  inflating: ./data/celeba_hq/train/female/174921.jpg  
  inflating: ./data/celeba_hq/train/female/174924.jpg  
  inflating: ./data/celeba_hq/train/female/174943.jpg  
  inflating: ./data/celeba_hq/train/female/174962.jpg

In [ ]:
import shutil
path = '/content/data/celeba_hq/train/male/'
for i in range(len(Add)):
  ages = os.listdir('Face-hq')
  age = (int)(Age[i])
  add = path+Add[i]
  # creates folder 
  if str(age) not in ages:
    print(ages)
    os.mkdir('Face-hq/{}'.format(age))
  dest = 'Face-hq/{}/{}.png'.format(age, i) 
  shutil.move(add, dest)  
  



[]
['33']
['33', '24']
['33', '24', '28']
['33', '24', '28', '39']
['33', '24', '28', '63', '39']
['52', '33', '24', '28', '63', '39']
['52', '25', '33', '24', '28', '63', '39']
['66', '52', '25', '33', '24', '28', '63', '39']
['66', '52', '56', '25', '33', '24', '28', '63', '39']
['66', '30', '52', '56', '25', '33', '24', '28', '63', '39']
['66', '30', '52', '53', '56', '25', '33', '24', '28', '63', '39']
['66', '30', '52', '53', '56', '25', '45', '33', '24', '28', '63', '39']
['66', '30', '52', '53', '56', '25', '45', '33', '24', '28', '63', '39', '17']
['66', '30', '31', '52', '53', '56', '25', '45', '33', '24', '28', '63', '39', '17']
['66', '30', '31', '52', '53', '56', '44', '25', '45', '33', '24', '28', '63', '39', '17']
['66', '30', '35', '31', '52', '53', '56', '44', '25', '45', '33', '24', '28', '63', '39', '17']
['66', '30', '35', '31', '52', '53', '56', '44', '25', '45', '33', '24', '28', '63', '40', '39', '17']
['21', '66', '30', '35', '31', '52', '53', '56', '44', '25', '

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! cp -r Face-hq 'drive/My Drive'